![alt text](pandas.png "Title")

In [2]:
import pandas as pd
import random

# Data aggregations and grouping

## Test data

In [3]:
# A simple demographic df
rows = 10
countries = ['USA', 'Switzerland', 'Belgium']
genders = ['Male', 'Female']
rawdata = {
    'usubjid': ['study_A_' + str(n) for n in range(1000, 1000 + rows)],
    'age'    : [random.randint(20, 50) for n in range(rows)], # randint() gives a random number within a range       
    'bmi'    : [random.randint(15, 35) for n in range(rows)],        
    'gender' : [random.choice(genders) for n in range(rows)], # choice() picks a value from an iterator
    'country': [random.choice(countries) for n in range(rows)] 
}

dm = pd.DataFrame(rawdata)

# look at the first 5 rows
dm.head()

,usubjid,age,bmi,gender,country
0,study_A_1000,30,29,Female,Belgium
1,study_A_1001,44,33,Female,Belgium
2,study_A_1002,44,20,Male,Belgium
3,study_A_1003,24,17,Female,USA
4,study_A_1004,45,21,Male,USA


## Data aggregation

A Dataframe object has many aggregation methods. They return a Series where the index holds the used df variable names

In [4]:
# Calculate the mean values of all numeric variables in the df:
dm.mean()

# many operations are available, e.g. count, sum, median, std, min, max, first, last

/shared/R_Temp/ndupuis/ipykernel_2915708/1687557877.py:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  dm.mean()


age    34.4
bmi    23.3
dtype: float64

In [8]:
# All combined in one shot:
df = dm.describe()
df


,age,bmi
count,10.000000,10.000000
mean,34.400000,23.300000
std,10.606078,6.129165
min,22.000000,15.000000
25%,24.250000,20.000000
50%,36.000000,22.000000
75%,44.000000,27.500000
max,46.000000,33.000000


In [5]:
# Some methods can work on strings too (lexicograph order)
dm.max()

usubjid    study_A_1009
age                  50
bmi                  32
gender             Male
country             USA
dtype: object

In [6]:
# You can also use agg() and pass a function name or a list of function names:
dm.agg(['median','min'])

,usubjid,age,bmi,gender,country
min,study_A_1000,25.0,17.0,Female,Belgium
median,NaN,42.0,24.5,NaN,NaN


In [12]:
dm['age'].mean()

34.4

## Group by

In [7]:
# groupby() creates groups by variable values. No need to sort df beforehand.
# It returns a 'groupby' object, which also has many aggregation methods.

# let's see the mean values for all numeric variables, but this time by gender:
dm.groupby('gender').mean()

,age,bmi
gender,,
Female,44.333333,28.666667
Male,39.142857,23.857143


In [8]:
# The index was labelled after the group names, but maybe you don't want a labelled index:
dm.groupby(['gender'], as_index=False).mean()

,gender,age,bmi
0,Female,44.333333,28.666667
1,Male,39.142857,23.857143


In [9]:
# Multiple groups:
dm.groupby(['gender','country']).mean()

age        bmi
gender country                          
Female Belgium      46.000000  32.000000
       USA          43.500000  27.000000
Male   Belgium      36.333333  24.333333
       Switzerland  41.666667  24.000000
       USA          40.000000  22.000000

In [10]:
# We can apply groupby() on a Series:
grouped = dm['age'].groupby(dm['gender'])

# mean() still returns a Series
grouped.mean()

gender
Female    44.333333
Male      39.142857
Name: age, dtype: float64

In [13]:
# The 'group_by' object supports iteration, it returns a tuple containing the group name and the data:
for name, data in dm.groupby(['gender']):
    print('Group name=', name) # that's a string
    print(data, '\n')         # and that's a dataframe

Group name= Female
        usubjid  age  bmi  gender      country  mean  age_mean
0  study_A_1000   30   29  Female      Belgium  34.4      34.4
1  study_A_1001   44   33  Female      Belgium  34.4      34.4
3  study_A_1003   24   17  Female          USA  34.4      34.4
7  study_A_1007   25   23  Female  Switzerland  34.4      34.4
9  study_A_1009   42   20  Female          USA  34.4      34.4 

Group name= Male
        usubjid  age  bmi gender  country  mean  age_mean
2  study_A_1002   44   20   Male  Belgium  34.4      34.4
4  study_A_1004   45   21   Male      USA  34.4      34.4
5  study_A_1005   22   23   Male  Belgium  34.4      34.4
6  study_A_1006   22   15   Male  Belgium  34.4      34.4
8  study_A_1008   46   32   Male  Belgium  34.4      34.4 



/shared/R_Temp/ndupuis/ipykernel_2915708/1691012661.py:2: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for name, data in dm.groupby(['gender']):


In [14]:
# We can also group data by applying a function on the index.

# let's set an index first:
dm1 = dm.set_index(['country'])
dm1


,usubjid,age,bmi,gender,mean,age_mean
country,,,,,,
Belgium,study_A_1000,30,29,Female,34.4,34.4
Belgium,study_A_1001,44,33,Female,34.4,34.4
Belgium,study_A_1002,44,20,Male,34.4,34.4
USA,study_A_1003,24,17,Female,34.4,34.4
USA,study_A_1004,45,21,Male,34.4,34.4
Belgium,study_A_1005,22,23,Male,34.4,34.4
Belgium,study_A_1006,22,15,Male,34.4,34.4
Switzerland,study_A_1007,25,23,Female,34.4,34.4
Belgium,study_A_1008,46,32,Male,34.4,34.4


In [15]:
# Group by length of 'country' and show the max values in numeric variables:
dm1[['age','bmi']].groupby(len).max()

,age,bmi
country,,
3,45,21
7,46,33
11,25,23


## More sophistication

Let's create a Age mean by gender and add it as a new column. 

In [18]:
# Remember apply() ? It works on groupby objects too.
# We could use mean() directly instead but apply() gives more flexibility, especially combined with a lambda function.

# this creates a Series (because we subsetted dm to keep only the age column)
age_mean = dm.groupby('gender')['age'].apply(lambda group: group.mean())
age_mean

# Does the same: age_mean = dm.groupby('gender')['age'].mean()

gender
Female    33.0
Male      35.8
Name: age, dtype: float64

In [19]:
# Before we can add this on dm, the Series and the Dataframe must share the same index. 
dm2 = dm.set_index('gender')
dm2

,usubjid,age,bmi,country,mean,age_mean
gender,,,,,,
Female,study_A_1000,30,29,Belgium,34.4,34.4
Female,study_A_1001,44,33,Belgium,34.4,34.4
Male,study_A_1002,44,20,Belgium,34.4,34.4
Female,study_A_1003,24,17,USA,34.4,34.4
Male,study_A_1004,45,21,USA,34.4,34.4
Male,study_A_1005,22,23,Belgium,34.4,34.4
Male,study_A_1006,22,15,Belgium,34.4,34.4
Female,study_A_1007,25,23,Switzerland,34.4,34.4
Male,study_A_1008,46,32,Belgium,34.4,34.4


In [20]:
# and now we can add the Series easily onto the Dataframe
dm2['age_mean'] = age_mean

dm2.sort_index()

,usubjid,age,bmi,country,mean,age_mean
gender,,,,,,
Female,study_A_1000,30,29,Belgium,34.4,33.0
Female,study_A_1001,44,33,Belgium,34.4,33.0
Female,study_A_1003,24,17,USA,34.4,33.0
Female,study_A_1007,25,23,Switzerland,34.4,33.0
Female,study_A_1009,42,20,USA,34.4,33.0
Male,study_A_1002,44,20,Belgium,34.4,35.8
Male,study_A_1004,45,21,USA,34.4,35.8
Male,study_A_1005,22,23,Belgium,34.4,35.8
Male,study_A_1006,22,15,Belgium,34.4,35.8


Another example: let's select and sort the 3 highest bmi by gender.

In [21]:
dm

,usubjid,age,bmi,gender,country,mean,age_mean
0,study_A_1000,30,29,Female,Belgium,34.4,34.4
1,study_A_1001,44,33,Female,Belgium,34.4,34.4
2,study_A_1002,44,20,Male,Belgium,34.4,34.4
3,study_A_1003,24,17,Female,USA,34.4,34.4
4,study_A_1004,45,21,Male,USA,34.4,34.4
5,study_A_1005,22,23,Male,Belgium,34.4,34.4
6,study_A_1006,22,15,Male,Belgium,34.4,34.4
7,study_A_1007,25,23,Female,Switzerland,34.4,34.4
8,study_A_1008,46,32,Male,Belgium,34.4,34.4
9,study_A_1009,42,20,Female,USA,34.4,34.4


In [13]:
def highest_bmi(group, select=3):
    return group.sort_values(by='bmi', ascending=False)[-select:]
    
dm.groupby('gender').apply(highest_bmi)

# or alternatively, if you like lambdas:
dm.groupby('gender').apply(lambda group: group.sort_values(by='bmi', ascending=False)[-3:])

usubjid  age  bmi  gender      country
gender                                               
Female 4  study_A_1004   24   18  Female          USA
       3  study_A_1003   36   17  Female          USA
       9  study_A_1009   40   16  Female          USA
Male   1  study_A_1001   28   35    Male      Belgium
       0  study_A_1000   41   25    Male  Switzerland
       8  study_A_1008   49   22    Male          USA

__________________________________________________
Nicolas Dupuis, Methodology and Innovation (IDAR C&SP), 2020+